## Segmenting and Clustering Neighborhoods in Toronto

In [11]:
from bs4 import BeautifulSoup
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import urllib # lib to handle url

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [12]:
#Having the url where the Zip table come from
wikiurl = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [14]:
#To scrape all content from wikiurl
wikiResponse = urllib.request.urlopen(wikiurl)
wikiHtmlDoc = wikiResponse.read()

In [15]:
#Use bs4 to obtain the table of data
wikiSoup = BeautifulSoup(wikiHtmlDoc,'html.parser')
wikiTable = wikiSoup.find('table',{'class':'wikitable sortable'})

In [16]:
#Read table into Pandas
table_rows = wikiTable.find_all('tr') #get every table row
data = []
for row in table_rows:
    td=[]
    for t in row.find_all('td'): # in each row, find the table data
        td.append(t.text.strip())
    data.append(td)
dfToronto = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighborhood'])

In [17]:
dfToronto.drop(dfToronto[dfToronto.Borough == 'Not assigned'].index, inplace=True) # remove not assigned Borough
dfToronto.drop(dfToronto[dfToronto['Borough'].isnull()].index,inplace=True) #remove the empty line
dfToronto = dfToronto.groupby(['PostalCode','Borough'])['Neighborhood'].apply(lambda x: ','.join(x)).reset_index() #combine neighborhood
dfToronto['Neighborhood'].replace('Not assigned',dfToronto['Borough'],inplace=True) # replace the empty neighborhood with borough name

In [18]:
dfToronto.head(12)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [19]:
dfToronto.shape

(103, 3)

## Part 2: Get geographical coordinates

In [ ]:
#!pip install geocoder

In [25]:
dfGeoToronto = pd.read_csv('Geo_Coordinates_Toronto.csv')
dfGeoToronto.rename(columns={'Postal Code':'PostalCode'},inplace=True)
dfGeoToronto.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [27]:
dfTorontoWithGeo = pd.merge(dfToronto,dfGeoToronto,how='outer',on='PostalCode',left_index=True)
dfTorontoWithGeo.head(12)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848
